### Scrape User Activity List

In [ ]:
# def scrapeTable(url, attempt=1):
#     if attempt == 4:
#         return pd.DataFrame(columns=['title', 'type', 'year', 'avg', 'status', 
#                                      'eps', 'times_watched', 'rating'])
    
#     resp = requests.get(f'http://192.168.0.3:5000/special-requests?url={quote(url)}')
#     if resp.text != '':
#         try:
#             soup = BeautifulSoup(resp.text, 'html.parser')
#             table = soup.find('table')
#             chunk = pd.read_html(StringIO(str(table)))[0]
#             chunk.columns = ['title', 'type', 'year', 'avg', 'status', 'eps', 'times_watched', 'rating']
#             chunk['url'] = [np.where(tag.has_attr('href'), 
#                                'https://www.anime-planet.com' + tag.get('href'), 
#                                'no link') for tag in [td.find('a') for td in table.find_all('td', attrs={'class':'tableTitle'})]]
#             chunk['url'] = chunk['url'].astype('string')
#             return chunk
#         except:
#             return scrapeTable(url, attempt+1)
#     else:
#         return scrapeTable(url, attempt+1)

# def scrapeUserWatched(username, attempt=1):
#     if attempt == 4:
#         return pd.DataFrame(columns=['title', 'type', 'year', 'avg', 'status', 
#                                      'eps', 'times_watched', 'rating', 'username'])
    
#     url = f'https://www.anime-planet.com/users/{username}/anime/watched?sort=title&mylist_view=list'
#     resp = requests.get(f'http://192.168.0.3:5000/special-requests?url={quote(url)}')
#     if resp.text != '':
#         try:
#             soup = BeautifulSoup(resp.text, 'html.parser')
#             ul = soup.find('ul', attrs={'class':'nav'})
#             page_nums = []
#             for tag in ul.find_all('a'):
#                 try:
#                     page_nums.append(int(tag.text))
#                 except:
#                     continue
#             num_pages = max(page_nums)

#             urls = [f'{url}&page={i}' for i in range(1, num_pages+1)]

#             with ThreadPoolExecutor(max_workers=10) as executor:
#                 df_list = list(executor.map(scrapeTable, urls))

#             df = pd.concat(df_list)

#             df['username'] = username
            
#             with db.connect() as con:
#                 df.to_sql('user', con, index=False, if_exists='append', method='multi')
            
#         except:
#             scrapeUserWatched(username, attempt+1)
        
#     else:
#         scrapeUserWatched(username, attempt+1)

# for username in tqdm(usernames):
#     scrapeUserWatched(username)

In [ ]:
url_html_df = pd.DataFrame(columns=['url','html_text'])
pending = set()

In [ ]:
def getUserFirstPage(username, attempt=1):
    if attempt == 4:
        return (url, '')
    
    try:
        url = f'https://www.anime-planet.com/users/{username}/anime?sort=title&mylist_view=list'
        resp = requests.get(f'http://192.168.0.3:5000/special-requests?url={quote(url)}')
        if resp.text != '':
            return (url, resp.text)
        
        else:
            return getUserFirstPage(username, attempt+1)
            
    except:
        return getUserFirstPage(username, attempt+1)   

In [ ]:
def addAdditionalPages(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    ul = soup.find('ul', attrs={'class':'nav'})
    page_nums = []
    for tag in ul.find_all('a'):
        try:
            page_nums.append(int(tag.text))
        except:
            continue
    num_pages = max(page_nums)

    urls = [f'{url}&page={i}' for i in range(1, num_pages+1)]
    
    pending.update(urls)

In [ ]:
chunksize = 10

username_chunks = chunker(usernames, chunksize)

for idx, username_chunk in enumerate(tqdm(username_chunks, total=len(usernames)/chunksize)):
    with ThreadPoolExecutor(max_workers=10) as executor:
        list_of_tups = list(executor.map(getUserFirstPage, username_chunk))
        url_html_df = pd.concat([url_html_df, 
                                 pd.DataFrame(list_of_tups, columns=['url','html_text'])], 
                                ignore_index=True)
    if idx % 10 == 0:
        
    time.sleep(random.randint(2, 5))